In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
#OS
import os

#Data handling
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


Data Processing

In [ ]:
train_ds = 'data/Training/'
test_ds = 'data/Testing/'

train_paths= []
train_labels = []